
- Reddit, AI subredit: https://www.reddit.com/r/artificial/ 

-- Description: r/artificial is the largest subreddit dedicated to all issues related to Artificial Intelligence or AI

-- Community size: 92.3K members 
-- Available tool to parse reddit: https://praw.readthedocs.io/en/latest/tutorials/comments.html

-- to obtain a year of historical reddits we can use 
https://towardsdatascience.com/scrape-reddit-data-using-python-and-google-bigquery-44180b579892

https://minimaxir.com/2015/10/reddit-bigquery/

pushShift API https://medium.com/@RareLoot/using-pushshifts-api-to-extract-reddit-submissions-fb517b286563


In [517]:
!pip install praw
!pip install psaw
#! usr/bin/env python3
import praw #Python Reddit API Wrapper
import pandas as pd
import datetime as dt
from psaw import PushshiftAPI



You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [518]:
%run "All_helper_classes.ipynb" sort=False

#from "Models training helpers.ipynb" we will import 3 helper classes and the associated helper methods.
parsing_helpers = generic_parsing_helpers()
aivm_helper = AIVM_helper()
model_helpers, info = default_model_helpers_for_project()
#if you want to see the paths now can call with...
paths = project_paths()


Necessary libraries imported.
Initialised generic_parsing_helpers class and methods
Initialised AIVM_helper class and methods
Initialised model_helpers class and methods


In [519]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;}

<IPython.core.display.Javascript object>

In [520]:
credentials = {'REDDIT_USER_NAME':'unmakingyou',
               'REDDIT_PASSWORD':'*******',
               'APP_NAME' : 'AIVC',
               'PERSONAL_USE_SCRIPT':'LEvS54694dQuMg',
              'SECRET_KEY' :'J5Z28cV8NTT9sohzpUk-G-dtVUo'}

In [498]:
class SubredditScraper:


    def __init__(self, sub, credentials, sort='new', lim=900, mode='w'):
        print (credentials)
        self.sort = sort
        self.lim = lim
        self.mode = mode
        self.subreddit = sub
        self.credentials = credentials
        
        
        self.REDDIT_USER_NAME = self.credentials['REDDIT_USER_NAME']
        self.REDDIT_PASSWORD = self.credentials['REDDIT_PASSWORD']
        self.APP_NAME = self.credentials['APP_NAME']
        self.PERSONAL_USE_SCRIPT = self.credentials['PERSONAL_USE_SCRIPT']
        self.SECRET_KEY = self.credentials['SECRET_KEY']
        print(
            f'SubredditScraper instance created with values '
            f'sub = {sub}, sort = {sort}, lim = {lim}, mode = {mode}')
        
        self.reddit = praw.Reddit(client_id= self.PERSONAL_USE_SCRIPT,
                                  client_secret= self.SECRET_KEY , 
                                  user_agent= self.APP_NAME, 
                                  username=self.REDDIT_USER_NAME, 
                                  password=self.REDDIT_PASSWORD)
        self.api = PushshiftAPI(self.reddit)
        
    def get_submissions(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit(self.subreddit).new(limit=self.lim), self.api
        elif self.sort == 'top':
            return self.sort, reddit.subreddit(self.subreddit).top(limit=self.lim), self.api
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit(self.subreddit).hot(limit=self.lim), self.api
        elif self.sort == 'controversial':
            return self.sort, reddit.subreddit(self.subreddit).controversial(limit=self.lim), self.api
        elif self.sort == 'rising':
            return self.sort, reddit.subreddit(self.subreddit).rising(limit=self.lim), self.api
        elif self.sort == 'gilded':
            return self.sort, reddit.subreddit(self.subreddit).gilded(limit=self.lim), self.api 
        else:
            self.sort = 'hot'
            print('Sort method  not recognized, defaulting to hot.')
            return self.sort, reddit.subreddit(self.sub).hot(limit=self.lim), self.api
        
    def clean_description (self, text):
        #https://regex101.com/r/cO8lqs/2
        #remove html markup
        text = re.sub("(<.*?>)","",text)
        text = re.sub("[\|\[].*?[\|\]]", "", text)
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE)
        text = text.replace("-", " ")  
      
        text = html.unescape(text)
        text = text.replace('\n', ' ').replace('\r', '').replace('\'', '"').replace('"', "'")
        text = text.strip()
        text = re.sub(' +', ' ', text)
        text = re.sub(' \.+', ' ', text)
       
        text = re.sub('<', '', text)
        text = re.sub('>', '', text)
        text = text.replace(u'\xa0', u' ')
        
       
        
        return text
    

# Create scraper class with Subreddit and Credentials defined
r/artificial is the largest subreddit dedicated to all issues related to Artificial Intelligence or AI. What does that mean? That is actually a tricky question, as the definition of AI is a topic of hot debate among people both inside and outside of the field. Broadly speaking, it is about machines that behave intelligently in some way, but this means different things to different people.
    Note: There are several sorts that we can use to iterate through the submissions: 
        - .hot, 
        - .new, 
        - .controversial, 
        - .top, 
        - .rising,
        - .gilded. 
        
We can also use .search("SEARCH_KEYWORDS") to get only results matching an engine search.

In [514]:
subreddit_scrape = SubredditScraper('artificial', credentials, sort='hot', lim=1000, mode='w')

{'REDDIT_USER_NAME': 'unmakingyou', 'REDDIT_PASSWORD': 'aid00rk5', 'APP_NAME': 'AIVC', 'PERSONAL_USE_SCRIPT': 'LEvS54694dQuMg', 'SECRET_KEY': 'J5Z28cV8NTT9sohzpUk-G-dtVUo'}
SubredditScraper instance created with values sub = artificial, sort = hot, lim = 1000, mode = w


# Obtain 'Submission' Objects
once we have a Subreddit instance, we can iterate through some of its submissions, each bound to an instance of Submission. 

    *Note: Carlos was proposing to only gather posts and first level answers
    *Note: Reddit’s request limit* is 1000 (work-around for this by querying the subreddits by date)

In [515]:
sort, submissions, psaw_api = subreddit_scrape.get_submissions()
display (sort, submissions, api)

'hot'

In [516]:
topics_dict = { "id":[],
                "title":[],
                "comment_num": [], 
                "comments":[],
                "score":[],   
                "created": [], 
                "body":[],
                 
              }

for submission in submissions:
    topics_dict["id"].append(submission.id)
    topics_dict["title"].append(submission.title)
    topics_dict["comment_num"].append(submission.num_comments)
    topics_dict["score"].append(submission.score)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

# Obtain a 'CommentForrest' object per submission
With a submission object we can then interact with its CommentForest through the submission’s comments attribute.
    - A CommentForest is a list of top-level comments each of which contains a CommentForest of replies.

In [392]:
for _id, x in zip(topics_dict["id"], range(len(topics_dict["id"])))  :
    submission = reddit.submission(id= _id)
#     print ("************",x)
    #all 'MoreComments' objects will be replaced until there are none left, as long as they satisfy the threshold.
    submission.comments.replace_more(limit=None)
    text = topics_dict["title"][x] + '.'
    for top_level_comment in submission.comments:
        #the body of the top level comments
        text = text + " " + top_level_comment.body
        for second_level_comment in top_level_comment.replies:
            text = text + " " + second_level_comment.body
    text = subreddit_scrape.clean_description(text)
    topics_dict["comments"].append(text)
#     display (text) print ("")

In [395]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

timestamp = reddit_data_pd["created"].apply(get_date)
reddit_data_pd['created'] = timestamp

In [397]:
print (len(reddit_data_pd.id.unique()))

25


In [398]:
reddit_data_pd.to_csv( "./reddit_data_rising.csv", index = None, header=True)

In [551]:
data1 = pd.read_csv("./reddit_data_new.csv") 
data2 = pd.read_csv("./reddit_data_top.csv")
data3 = pd.read_csv("./reddit_data_hot.csv")
data4 = pd.read_csv("./reddit_data_contro.csv")
data5 = pd.read_csv("./reddit_data_rising.csv")
total_recent = pd.concat([data1, data2, data3, data4, data5])
print (total_recent.shape[0])
print (len(total_recent.id.unique()))
total_recent.to_csv( "./reddit_data_all_recent.csv", index = None, header=True)

4013
2826


In [400]:
display(total.head(10))

,id,title,comment_num,comments,score,created,body
0,e3jfe3,A list of Monte Carlo tree search research pap...,0,A list of Monte Carlo tree search research pap...,1,2019-11-30 04:24:18,&#x200B;\n\nhttps://preview.redd.it/vie7kcgzeo...
1,e3i7ff,Question about AI from a newbie,2,Question about AI from a newbie. The results d...,1,2019-11-30 03:00:49,Hi guys. I dont have any experience on program...
2,e3i7dk,Former Go champion beaten by DeepMind retires ...,1,Former Go champion beaten by DeepMind retires ...,2,2019-11-30 03:00:43,NaN
3,e3hv3m,Critical AI Algorithms Are Not Secure (Yet),0,Critical AI Algorithms Are Not Secure (Yet).,1,2019-11-30 02:36:02,NaN
4,e3hq03,Tencent Open-Sources High-Performance Graph Co...,0,Tencent Open Sources High Performance Graph Co...,1,2019-11-30 02:25:51,NaN
5,e3h9fv,Advice on Research prospects,0,Advice on Research prospects.,1,2019-11-30 01:54:45,What are the research prospects at the interse...
6,e3h503,Here is a good list of AI experts in Europe to...,0,Here is a good list of AI experts in Europe to...,4,2019-11-30 01:45:40,NaN
7,e3h2yy,How can AI be used to categorize emails?,0,How can AI be used to categorize emails?.,0,2019-11-30 01:41:23,Through text-based classification or natural l...
8,e3feph,Facebook Secretly Built a Facial Recognition A...,3,Facebook Secretly Built a Facial Recognition A...,26,2019-11-29 23:28:42,NaN
9,e3eykd,What if we make an GAI give it access to the i...,6,What if we make an GAI give it access to the i...,0,2019-11-29 22:48:52,The idea is we make some super intelligent AI ...


In [401]:
display(total.iloc[0:1])
display(total.iloc[0][3])

,id,title,comment_num,comments,score,created,body
0,e3jfe3,A list of Monte Carlo tree search research pap...,0,A list of Monte Carlo tree search research pap...,1,2019-11-30 04:24:18,&#x200B;\n\nhttps://preview.redd.it/vie7kcgzeo...


# Historic Reddits using psaw api

In [530]:
subreddit_scrape = SubredditScraper('artificial', credentials, sort='hot', lim=1000, mode='w')

sort, submissions, psaw_api = subreddit_scrape.get_submissions()
display (sort, submissions, api)

start_epoch=int(dt.datetime(2019, 9, 2).timestamp()) #Year, month, day
end_epoch=int(dt.datetime(2019, 10, 1).timestamp())
print (start_epoch)

submissions = list(api.search_submissions(before=end_epoch,
                                          after=start_epoch,
                            subreddit='artificial',
                            filter=['url','author', 'title', 'subreddit'],
                            limit=1000))


topics_dict = { "id":[],
                "title":[],
                "comment_num": [], 
                "comments":[],
                "score":[],   
                "created": [], 
                "body":[],
                 
              }

for submission in submissions:
    topics_dict["id"].append(submission.id)
    topics_dict["title"].append(submission.title)
    topics_dict["comment_num"].append(submission.num_comments)
    topics_dict["score"].append(submission.score)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
print (topics_dict["title"])

for _id, x in zip(topics_dict["id"], range(len(topics_dict["id"])))  :
    submission = reddit.submission(id= _id)
#     print ("************",x)
    #all 'MoreComments' objects will be replaced until there are none left, as long as they satisfy the threshold.
    submission.comments.replace_more(limit=None)
    text = topics_dict["title"][x] + '.'
    for top_level_comment in submission.comments:
        #the body of the top level comments
        text = text + " " + top_level_comment.body
        for second_level_comment in top_level_comment.replies:
            text = text + " " + second_level_comment.body
    text = subreddit_scrape.clean_description(text)
    topics_dict["comments"].append(text)
#     display (text) 
#     print ("")
    
    

reddit_data_pd = pd.DataFrame(topics_dict)
timestamp = reddit_data_pd["created"].apply(get_date)
reddit_data_pd['created'] = timestamp

display (reddit_data_pd.head())
display (reddit_data_pd.shape[0])
reddit_data_pd.to_csv( "./reddit_data_historic_Sep_2019.csv", index = None, header=True)

{'REDDIT_USER_NAME': 'unmakingyou', 'REDDIT_PASSWORD': 'aid00rk5', 'APP_NAME': 'AIVC', 'PERSONAL_USE_SCRIPT': 'LEvS54694dQuMg', 'SECRET_KEY': 'J5Z28cV8NTT9sohzpUk-G-dtVUo'}
SubredditScraper instance created with values sub = artificial, sort = hot, lim = 1000, mode = w


'hot'

1567375200
['AI isn’t funny anymore', 'What is a Chatbot -Top Ways AI is Impacting the Mobile Development Industry - Kontorara', 'Peter Norvig - Artificial Intelligence: A Modern Approach | Artificial Intelligence Podcast', 'Picture Perfect Beauty Courtesy AI Makeup Artist', "Spot's launch is a reality check on the current state of AI/robotics", 'Songwriting (music and lyrics), music mastering, video and more created by AI', 'A Warning From Ron Swanson [DeepFake Voice]', 'Top 5 Future Applications of Artificial Intelligence', 'Hi ! I just wrote an article about the Dual Shot Face Detector, which was presented in CVPR 2019. What do you think of it?', 'Study: Artificial intelligence can diagnose diseases with impressive accuracy. 14 studies suggest that deep learning systems correctly detect a disease 87 percent of the time, while healthcare professionals are accurate 86 percent of the time.', 'Beautiful vid on connections between the different types of machine learning and evolution.', 

,id,title,comment_num,comments,score,created,body
0,dbhpys,AI isn’t funny anymore,2,AI isn’t funny anymore. We probably need to st...,3,2019-10-01 06:27:47,
1,dbfyex,What is a Chatbot -Top Ways AI is Impacting th...,0,What is a Chatbot Top Ways AI is Impacting the...,2,2019-10-01 04:31:43,
2,dbfe2z,Peter Norvig - Artificial Intelligence: A Mode...,1,Peter Norvig Artificial Intelligence: A Modern...,12,2019-10-01 03:54:22,
3,dbeyst,Picture Perfect Beauty Courtesy AI Makeup Artist,0,Picture Perfect Beauty Courtesy AI Makeup Artist.,3,2019-10-01 03:26:21,
4,dbdwey,Spot's launch is a reality check on the curren...,0,Spot's launch is a reality check on the curren...,5,2019-10-01 02:14:15,Boston Dynamics launched its flagship Spot rob...


583

In [552]:
data1 = pd.read_csv("./reddit_data_historic_Jan_2019.csv") 
data2 = pd.read_csv("./reddit_data_historic_Feb_2019.csv")
data3 = pd.read_csv("./reddit_data_historic_Mar_2019.csv")
data4 = pd.read_csv("./reddit_data_historic_Apr_2019.csv")
data5 = pd.read_csv("./reddit_data_historic_May_2019.csv")
data6 = pd.read_csv("./reddit_data_historic_Jun_2019.csv")
data7 = pd.read_csv("./reddit_data_historic_Jul_2019.csv")
data8 = pd.read_csv("./reddit_data_historic_Aug_2019.csv")
data9 = pd.read_csv("./reddit_data_historic_Sep_2019.csv")

total = pd.concat([data1, data2, data3, data4, data5,data6, data7, data8, data9, total_recent])
print ("recent len", total_recent.shape[0])
print ("recent unique", len(total_recent.id.unique()))
print ("total len",total.shape[0])
print ("total unique",len(total.id.unique()))

total.drop_duplicates(subset=None, keep='last', inplace=True)
total.to_csv( "./reddit_data_all.csv", index = None, header=True)
print (total.shape[0])
print (len(total.id.unique()))

total = total.loc[total['comment_num'] > 1]
print (total['comment_num'].unique())
print (total.shape[0])
display(total.head(10))
display(total.iloc[1:1])
display(total.iloc[1][3])

recent len 4013
recent unique 2826
total len 9550
total unique 7706
8534
7706
[ 12   5  30  34  10  11  13   3  20  14   2   4  37  15  17   7  19  71
   6  39   9  24   8  21  25  22  38  44 104 120  29  36  16  48  46  26
  47  41  31  18  23  35  40  72  59  32  28 112  52  49  53  27  92  50
  45  81  66  80  51  82  33  88 142  42  90  74  58  65  60  67  43  57
  87 128  54  73  62  61  99  55  83  56  84]
3660


,id,title,comment_num,comments,score,created,body
3,alrasv,Original video + Stennifer Lawrscemi,12,Original video + Stennifer Lawrscemi. There's ...,86,2019-02-01 01:37:36,[deleted]
11,alp4h8,Robot that can ‘imagine itself' built by scien...,5,Robot that can ‘imagine itself' built by scien...,12,2019-01-31 21:48:58,NaN
18,almwcp,Conditions on StarCraft Supremacy and fair pla...,30,Conditions on StarCraft Supremacy and fair pla...,13,2019-01-31 15:59:05,NaN
21,all1yw,What is the next branch of math which will be ...,34,What is the next branch of math which will be ...,37,2019-01-31 12:06:47,"We had logic, optimization and applied statist..."
22,alkklt,"Posted in /hardware, suggested I ask you guys....",10,"Posted in /hardware, suggested I ask you guys....",1,2019-01-31 11:12:00,This has to be something that's been considere...
23,alk978,Will someone humor me and ELI5 why Alexa is so...,11,Will someone humor me and ELI5 why Alexa is so...,2,2019-01-31 10:37:03,"Hello all,\n\n&#x200B;\n\nI'm perfectly willin..."
24,aliljx,A Step Closer to Self-Aware Machines,13,A Step Closer to Self Aware Machines. Interest...,44,2019-01-31 07:49:49,NaN
29,ald81z,Lastest machine learning methods applications ...,3,Lastest machine learning methods applications ...,7,2019-01-30 23:33:03,I wonder why none of the recent methods in rob...
40,alaftc,"IBM releases Diversity in Faces, a dataset wit...",20,"IBM releases Diversity in Faces, a dataset wit...",132,2019-01-30 16:43:41,NaN
43,al9di2,What if you set the goal of an AGI to earn mon...,14,What if you set the goal of an AGI to earn mon...,0,2019-01-30 14:15:48,Is there any way that could go wrong?


,id,title,comment_num,comments,score,created,body


"Robot that can ‘imagine itself' built by scientists. After digging around for a while, ( seems to be the article they're talking about. TL;DR: Show the robot thousands of different trajectories, train it to learn its kinematic structure, then use this model to complete a picking task (44% pick rate) and a hand writing task Very sensationalized article imo Yeah I was literally thinking after reading the headline, 'okay... What NN architecture did they use and how ling did it have to train on looking at pictures of itself?' 😂 nothing revolutionary, keep science writing off pop media Robots have been mapping environment around them for a while. About time, they figure out their dimensions and mechanical movement. wow that is a interesting development in robotics."

In [550]:
display(total.iloc[1:1])
display(total.iloc[3][3])
display(total.iloc[3][6])

,id,title,comment_num,comments,score,created,body


"What is the next branch of math which will be rebranded as AI?. wait hold up when did logic enter the picture? Good old fashioned AI Back in the 1850's, when people automated syllogistic logic and produced mechanical machines that could work through syllogism problems. you'll have to dig through the references here, and all, if you want any real understanding, but the info is there for you. ( It was what Liebniz drew upon when he made boolean logic a thing, and the substratum of logical operators that underwrites all of computer science except maybe binary storage of information and related theories. enjoy. See knowledge base, knowledge representation. What a loaded, sour question... Why rebranded? Something happened between Leibniz, Descartes & Co and now. I really hate statements like 'AI/ML is really just X', X = {Linear Algebra; Applied Statistics; } You could have asked the question more productively as: Which branch of math is most under utilised / overlooked in current AI resea

"We had logic, optimization and applied statistics/regression. What will be the next branch of math which we will call as part of AI?\n\n\nI'm suspecting applied category theory"

# Google BigQuery Data Reddit - downloaded but not used
https://towardsdatascience.com/scrape-reddit-data-using-python-and-google-bigquery-44180b579892

In [470]:
submissions = pd.read_csv("./submissions_0719.csv")
submissions1 = pd.read_csv("./comments_0719.csv")
comments = pd.read_csv("./comments.csv")
display (submissions.head())
display(submissions.iloc[0][1])
comments.drop_duplicates(subset=None, keep='first', inplace=True)
print (comments.shape[0])
print (len(comments.id.unique()))
comments = comments.drop(comments[comments.body =='[removed]'].index)
comments = comments.drop(comments[comments.body =='[deleted]'].index)
comments = comments.drop(['id'], axis = 1)

comments = comments.rename(columns={'parent_id': 'id'})
display (comments.head())
print (comments.shape[0])
grouped = comments.groupby('parent_id')['body']

display (grouped.tail(100))

new = submissions.join(comments.set_index('id')[['body']], on='id')
display (new.head())